# Action-Conditioned 3D Human Motion Synthesis with Transformer VAE (ACTOR)

ACTOR learns an action-aware latent representation for human motions by training a generative variational autoencoder (VAE). By sampling from this latent space and querying a certain duration through a series of positional encodings, ACTOR synthesizes variable-length motion sequences conditioned on a categorical action. Specifically, a Transformer-based architecture is designed, for encoding and decoding a sequence of parametric SMPL human body models estimated from action recognition datasets. 

## Dataset

To get the pre-process the dataset, please refer to the this [Github repository](https://github.com/Mathux/ACTOR) and agree to the license. There following code shows examples from `HumanAct12` dataset.

In [1]:
# Set data path
datapath = "F://research/ACTOR/data/HumanAct12Poses/"

In [2]:
# change the working directory to the source code folder
import os
os.chdir("../../../../src")
os.getcwd()

'E:\\researches\\GenMotion\\src'

## Training

In [3]:
from algorithm.action_conditioned.params import HumanAct12Params
from algorithm.action_conditioned.datasets.humanact12poses import HumanAct12Poses

In [4]:
# load parameters
parameters = vars(HumanAct12Params())

In [5]:
# get datasets
datasets = HumanAct12Poses(split="train", datapath=datapath, **parameters)
print("dataset length: ", len(datasets))

dataset length:  1190


In [8]:
# load model
assert parameters["modeltype"] == 'cvae'
assert parameters["archiname"] == "transformer"

from algorithm.action_conditioned.models.architectures.transformer import Encoder_TRANSFORMER, Decoder_TRANSFORMER
from algorithm.action_conditioned.models.modeltype.cvae import CVAE

encoder = Encoder_TRANSFORMER(**parameters)
decoder = Decoder_TRANSFORMER(**parameters)

parameters["outputxyz"] = "rcxyz" in parameters["lambdas"]

TypeError: __init__() missing 3 required positional arguments: 'njoints', 'nfeats', and 'num_classes'